In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [23]:
class Encoder(nn.Module):
    def __init__(self, bottleneck_dims=64):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, bottleneck_dims, 7)
        )

    def forward(self, x):
        return self.encoder(x)



In [24]:
class Decoder(nn.Module):
    def __init__(self, bottleneck_dims=64):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(bottleneck_dims, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)



In [25]:
class Autoencoder(nn.Module):
    def __init__(self, bottleneck_dims=64):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(bottleneck_dims=bottleneck_dims)
        self.decoder = Decoder(bottleneck_dims=bottleneck_dims)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



In [26]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)



In [13]:
def train(model, criterion, optimizer, train_loader, num_epochs=10, sigma=0.1):
    model.train()
    for epoch in range(num_epochs):
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit='batch') as pbar:
            for data in train_loader:
                img, _ = data
                noisy_img = img + torch.randn_like(img) * sigma
                noisy_img = torch.clamp(noisy_img, 0., 1.)  # clip to range [0, 1]
                optimizer.zero_grad()
                recon = model(noisy_img)
                loss = criterion(recon, img)
                loss.backward()
                optimizer.step()
                pbar.update(1)
                pbar.set_postfix({'Loss': loss.item()})

# Testing function
def test(model, test_loader, sigma=0.1):
    model.eval()
    ssim_score = 0
    total_images = 0
    with torch.no_grad():
        with tqdm(total=len(test_loader), desc=f"Testing", unit='batch') as pbar:
            for data in test_loader:
                img, _ = data
                noisy_img = img + torch.randn_like(img) * sigma
                noisy_img = torch.clamp(noisy_img, 0., 1.)  # clip to range [0, 1]
                recon = model(noisy_img)
                ssim_score += np.sum([ssim(img[i, 0].numpy(), recon[i, 0].numpy(), data_range=1) for i in range(img.shape[0])])
                total_images += img.shape[0]
                pbar.update(1)
                
    return ssim_score / total_images



In [8]:
autoencoder = Autoencoder()


In [15]:
sigmas = [0.1, 0.5, 1.0]  # Different sigma values
avg_ssim_scores = []
for sigma in sigmas:
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    train(autoencoder, criterion, optimizer, train_loader, num_epochs=10, sigma=sigma)
    avg_ssim_score = test(autoencoder, test_loader, sigma=sigma)
    avg_ssim_scores.append(avg_ssim_score)

print("Average SSIM test scores for different sigma values:", avg_ssim_scores)



Testing: 100%|██████████| 157/157 [00:06<00:00, 24.71batch/s]

Average SSIM test scores for different sigma values: [0.9698094446808017, 0.8700647332417271, 0.6852523276026179]


As, it can be seen that with increase in sigma, accuracy of the model is decreasing. This is because, as the noise increases, the model is not able to learn the underlying pattern in the data.

#### Task 2 (Varying Sigma):
- As the sigma value increases, the amount of noise added to the images during training also increases.
- With higher sigma values, the noise becomes more significant, affecting the reconstruction quality of the autoencoder.
- Consequently, the average SSIM test score tends to decrease as sigma increases, indicating lower similarity between the original and reconstructed images.

In [27]:
bottleneck_dims = [8, 16, 32]  # Different bottleneck dimensionality
avg_ssim_scores = []
sigma = 0.5  # Constant sigma value
for dim in bottleneck_dims:
    autoencoder = Autoencoder(dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    train(autoencoder, criterion, optimizer, train_loader, num_epochs=10)
    avg_ssim_score = test(autoencoder, test_loader)
    avg_ssim_scores.append(avg_ssim_score)

print("Average SSIM test scores for different bottleneck dimensionalities:", avg_ssim_scores)


Testing: 100%|██████████| 157/157 [00:06<00:00, 23.73batch/s]

Average SSIM test scores for different bottleneck dimensionalities: [0.7839948960005227, 0.8993986273019243, 0.9453540767481474]


With decreasing dimension of bottleneck, accuracy is decreasing. As, the bottleneck dimension decreases, the model is not able to learn the underlying pattern in the data. This is because, the model is not able to learn the important features in the data.